In [59]:
import pandas as pd
import numpy as np
import json
import gzip
import os
import os.path
from os import listdir
import multiprocessing
from tqdm import tqdm
import mmap
from multiprocessing import Pool
import seaborn as sns
import matplotlib.pyplot as plt
import py_entitymatching as em
import py_stringmatching as sm
import scipy
from scipy.io import arff
from sklearn import preprocessing
#from sklearn.model_selection import train_test_split
from skmultilearn.model_selection import iterative_train_test_split
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
import progressbar
import itertools

In [53]:
product_path = '../../../../src/data/product'
cluster_path = os.path.join(product_path, 'lspc2020_to_tablecorpus/Cleaned')
cleaned_top100_path = os.path.join(product_path, 'product_top100/cleaned')
top100_path = os.path.join(product_path, 'product_top100')
cleaned_min3_path = os.path.join(product_path, 'product_minimum3/cleaned')
min3_path = os.path.join(product_path, 'product_minimum3')
results_path = os.path.join(product_path, 'product_results/after_preprocessing')
train_test_input_path = os.path.join(product_path, 'train_test_split/input')

In [10]:
'''
entities = ['Bikes', 'Cars', 'Clothes', 'Drugstore', 'Electronics', 'Technology', 'Tools', 'Random']
count = 0
for entity in entities:
    df_category1 = pd.read_csv(os.path.join(mapping_corpus_path_2,
                                            '{}_clusters_all_8_tables_post_processed_lower_threshold.csv'.format(
                                                entity)))
    df_category1['category'] = entity.lower()
    if count == 0:
        df_concat = df_category1
    if count > 0:
        df_concat = pd.concat([df_category1, df_category2]).drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Valid'])
    df_category2 = df_concat
    count += 1
    
df_concat
'''

In [13]:
unique_clusters_df = pd.read_excel(os.path.join(results_path, 'Final_lower threshold.xlsx'))
unique_clusters = list(set(unique_clusters_df['cluster_id'].to_list()))

In [22]:
#df_all = df_concat[df_concat['cluster_id'].isin(unique_clusters)]
#df_all

In [26]:
'''
product_target = df_all['cluster_id']
product_data = df_all.drop(columns='cluster_id')

# create the train/test split
data_train, data_test, target_train, target_test = train_test_split(product_data, product_target, test_size=0.3, random_state=42, stratify=product_target)
'''

In [33]:
df_table_cleaned = df_cleaned[df_cleaned['table_id'] == 'Product_mountainbikespecialists.com_September2020.json.gz']
table = 'Product_mountainbikespecialists.com_September2020.json.gz'
valid_table_rows = df_table_cleaned['row_id'].to_list()
table_path = cleaned_min3_path
table_df = pd.read_json(os.path.join(table_path, '{}'.format(table)), compression='gzip', lines=True)

In [34]:
table_df

,row_id,name,offers,description,brand,page_url
0,0,Blowtorch,"{'gtin': '708752317437', 'availability': 'https://schema.org/OutOfStock', 'price': '30.00', 'pri...",Sometimes the simplest pieces can be a favorite piece of kit. The Blowtorch neck gaiter is that ...,{'name': '45NRTH'},https://www.mountainbikespecialists.com/product/45nrth-blowtorch-368817-1.htm
1,1,Jekyll 1,"[{'pricecurrency': 'USD', 'price': '8200.00', 'availability': 'https://schema.org/OutOfStock', '...","This precision instrument is for the enduro artists, the ones who will use its Gemini dual-trave...",{'name': 'Cannondale'},https://www.mountainbikespecialists.com/product/cannondale-jekyll-1-345682-1.htm
2,2,K1SL SuperLite Kool Wide Chain,"{'pricecurrency': 'USD', 'price': '25.00', 'gtin': '766759171056', 'availability': 'https://sche...","The KMC K1SL SuperLite Kool Chain is suitable for a variety of single-speed riding styles, such ...",{'name': 'KMC'},https://www.mountainbikespecialists.com/product/kmc-k1sl-superlite-kool-wide-chain-378124-1.htm
3,3,Corkscrew Bottle Opener,"{'pricecurrency': 'USD', 'price': '45.95', 'availability': 'https://schema.org/OutOfStock', 'gti...","In the tradition of our premium quality fold up tools, the BO-4 is a combination corkscrew and b...",{'name': 'Park Tool'},https://www.mountainbikespecialists.com/product/park-tool-corkscrew-bottle-opener-326868-1.htm
4,4,Mountain Sport Organic Disc Brake Pads (Shimano),"{'pricecurrency': 'USD', 'availability': 'https://schema.org/OutOfStock', 'price': '10.99', 'gti...","Jagwire Sport Organic pads feature an all-new resin compound. Mounted on a steel backing plate, ...",{'name': 'Jagwire'},https://www.mountainbikespecialists.com/product/jagwire-mountain-sport-organic-disc-brake-pads-s...
...,...,...,...,...,...,...
1935,1935,Challenge Large Tool Bag,None,The Brooks Challenge Large Tool Bag is a saddle bag for fitment to the rear of a saddle bearing ...,{'name': 'Brooks'},https://www.mountainbikespecialists.com/product/brooks-challenge-large-tool-bag-282611-1.htm
1936,1936,Femto Drive Rear,"{'pricecurrency': 'USD', 'price': '13.49', 'availability': 'https://schema.org/OutOfStock', 'gti...","Lezyne's Femto Drive Rear is a compact, bright, and beautifully made light. The lens is made of ...",{'name': 'Lezyne'},https://www.mountainbikespecialists.com/product/lezyne-femto-drive-rear-192883-1.htm
1937,1937,Impact Pro Women's Mountain Bike Shoe,"[{'pricecurrency': 'USD', 'gtin': '191529206285', 'price': '160.00', 'availability': 'https://sc...",The Impact Pro touts superior toe protection and quicker drying than its predecessor and utilize...,{'name': 'Five Ten'},https://www.mountainbikespecialists.com/product/five-ten-impact-pro-womens-mountain-bike-shoe-37...
1938,1938,Cool Mesh Super Light SL Base Layer,"{'pricecurrency': 'USD', 'gtin': '800550194491', 'availability': 'https://schema.org/OutOfStock'...",Craft's Superlight Mesh top features big-hole design for superior ventilation and cooling. • Mul...,{'name': 'Craft'},https://www.mountainbikespecialists.com/product/craft-cool-mesh-super-light-sl-base-layer-235278...


In [35]:
valid_table_rows

[1718, 1318]

In [36]:
table_df = table_df[table_df['row_id'].isin(valid_table_rows)]
table_df

,row_id,name,offers,description,brand,page_url
1318,1318,Treadwell Neo,"{'pricecurrency': 'USD', 'gtin': '884603786383', 'price': '2300.00', 'availability': 'https://sc...","An urban e-bike for daily rides to class or rehearsal, to the park or the studio, or to meet you...",{'name': 'Cannondale'},https://www.mountainbikespecialists.com/product/cannondale-treadwell-neo-366127-1.htm
1718,1718,Kids Trail Single-Speed 16-inch,"{'pricecurrency': 'USD', 'gtin': '884603785904', 'availability': 'https://schema.org/OutOfStock'...","Get your little riders rolling free and loving life on these high-performance, Kid-Correct bikes...",{'name': 'Cannondale'},https://www.mountainbikespecialists.com/product/cannondale-kids-trail-single-speed-16-inch-36336...


In [37]:
df_table_cleaned

,Unnamed: 0,Unnamed: 0.1,cluster_id,url,row_id,table_id,Valid,name,description,brand_x,brand_y,tokens
9,106,554015,1615000,https://www.mountainbikespecialists.com/product/cannondale-kids-trail-single-speed-16-inch-36336...,1718,Product_mountainbikespecialists.com_September2020.json.gz,1,kids trail single-speed 16-inch,"Get your little riders rolling free and loving life on these high-performance, Kid-Correct bikes...",NaN,NaN,"['kids', 'trail', 'single-speed', '16-inch']"
811,5460,24106971,1001195,https://www.mountainbikespecialists.com/product/cannondale-treadwell-neo-366127-1.htm,1318,Product_mountainbikespecialists.com_September2020.json.gz,1,treadwell neo,"An urban e-bike for daily rides to class or rehearsal, to the park or the studio, or to meet you...",NaN,NaN,"['treadwell', 'neo']"


In [43]:
new_table_df = pd.merge(table_df, df_table_cleaned[['row_id', 'cluster_id', 'table_id', 'brand_x', 'brand_y', 'tokens']], left_on='row_id', right_on='row_id', how='left')

In [44]:
new_table_df

,row_id,name,offers,description,brand,page_url,cluster_id,table_id,brand_x,brand_y,tokens
0,1318,Treadwell Neo,"{'pricecurrency': 'USD', 'gtin': '884603786383', 'price': '2300.00', 'availability': 'https://sc...","An urban e-bike for daily rides to class or rehearsal, to the park or the studio, or to meet you...",{'name': 'Cannondale'},https://www.mountainbikespecialists.com/product/cannondale-treadwell-neo-366127-1.htm,1001195,Product_mountainbikespecialists.com_September2020.json.gz,NaN,NaN,"['treadwell', 'neo']"
1,1718,Kids Trail Single-Speed 16-inch,"{'pricecurrency': 'USD', 'gtin': '884603785904', 'availability': 'https://schema.org/OutOfStock'...","Get your little riders rolling free and loving life on these high-performance, Kid-Correct bikes...",{'name': 'Cannondale'},https://www.mountainbikespecialists.com/product/cannondale-kids-trail-single-speed-16-inch-36336...,1615000,Product_mountainbikespecialists.com_September2020.json.gz,NaN,NaN,"['kids', 'trail', 'single-speed', '16-inch']"


In [45]:
new_table_df['category'] = 'bikes'
new_table_df

,row_id,name,offers,description,brand,page_url,cluster_id,table_id,brand_x,brand_y,tokens,category
0,1318,Treadwell Neo,"{'pricecurrency': 'USD', 'gtin': '884603786383', 'price': '2300.00', 'availability': 'https://sc...","An urban e-bike for daily rides to class or rehearsal, to the park or the studio, or to meet you...",{'name': 'Cannondale'},https://www.mountainbikespecialists.com/product/cannondale-treadwell-neo-366127-1.htm,1001195,Product_mountainbikespecialists.com_September2020.json.gz,NaN,NaN,"['treadwell', 'neo']",bikes
1,1718,Kids Trail Single-Speed 16-inch,"{'pricecurrency': 'USD', 'gtin': '884603785904', 'availability': 'https://schema.org/OutOfStock'...","Get your little riders rolling free and loving life on these high-performance, Kid-Correct bikes...",{'name': 'Cannondale'},https://www.mountainbikespecialists.com/product/cannondale-kids-trail-single-speed-16-inch-36336...,1615000,Product_mountainbikespecialists.com_September2020.json.gz,NaN,NaN,"['kids', 'trail', 'single-speed', '16-inch']",bikes


In [46]:
new_table_df.to_json(os.path.join(train_test_input_path, '{}'.format(entity.lower()), '{}'.format(table)), compression='gzip', orient='records', lines=True)

In [54]:
entities = ['Bikes', 'Cars', 'Clothes', 'Drugstore', 'Electronics', 'Technology', 'Tools', 'Random']

files_dict = {'minimum3': [file for file in os.listdir(min3_path) if file.endswith('.json.gz')], 
             'top100': [file for file in os.listdir(top100_path) if file.endswith('.json.gz')]}
    
for entity in entities:
    print(entity)
    df_category = pd.read_csv(os.path.join(cluster_path,
                                                '{}_clusters_all_8_tables_post_processed_lower_threshold.csv'.format(
                                                    entity)))
    df_cleaned = df_category[df_category['cluster_id'].isin(unique_clusters)]
    valid_tables = list(set(df_cleaned['table_id'].to_list()))
    
    count = 0
    with progressbar.ProgressBar(max_value=len(valid_tables)) as bar:
        for table in valid_tables:
            count += 1
            if table in files_dict['minimum3']:
                table_path = min3_path
            elif table in files_dict['top100']:
                table_path = top100_path
            else:
                print('no valid path for: {}'.format(table))
                continue

            df_table_cleaned = df_cleaned[df_cleaned['table_id'] == table]
            valid_table_rows = df_table_cleaned['row_id'].to_list()
            table_df = pd.read_json(os.path.join(table_path, '{}'.format(table)), compression='gzip', lines=True)
            table_df = table_df[table_df['row_id'].isin(valid_table_rows)]
            new_table_df = pd.merge(table_df, df_table_cleaned[['row_id', 'cluster_id', 'table_id', 'brand_x', 'brand_y', 'tokens']], left_on='row_id', right_on='row_id', how='left')
            new_table_df['category'] = entity.lower()
            new_table_df.to_json(os.path.join(train_test_input_path, '{}'.format(entity.lower()), '{}'.format(table)), compression='gzip', orient='records', lines=True)
            
            bar.update(count)
            

  0% (0 of 126) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--

Bikes


100% (126 of 126) |######################| Elapsed Time: 0:00:12 Time:  0:00:12


Cars


100% (234 of 234) |######################| Elapsed Time: 0:00:25 Time:  0:00:25


...

100% (476 of 476) |######################| Elapsed Time: 0:00:26 Time:  0:00:26


Tools


100% (9 of 9) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


Random


100% (2280 of 2280) |####################| Elapsed Time: 0:03:11 Time:  0:03:11


In [55]:
input_files_dict = {}
for entity in entities:
    train_test_category_path = os.path.join(train_test_input_path, '{}'.format(entity.lower()))
    input_files_dict[entity.lower()] = [file for file in os.listdir(train_test_category_path) if file.endswith('.json.gz')]

In [66]:
# try to find overlap between two categories:
combinations_list = []
for pair in itertools.combinations(entities,2):
    combinations_list.append(pair)
    
overlap_dict = {}
for pair in combinations_list:
    overlap = list(set(input_files_dict[pair[0].lower()]) & set(input_files_dict[pair[1].lower()]))
    overlap_dict[str(pair)] = overlap
    
overlap_dict

# save overlap_dict
with open(os.path.join(train_test_input_path, 'all/overlap_tables_between_categories_dict.json'), 'w', encoding='utf-8') as f:
    json.dump(overlap_dict, f)

In [67]:
# save all tables in one path
for entity in entities:
    print(entity)
    train_test_category_path = os.path.join(train_test_input_path, '{}'.format(entity.lower()))
    train_test_all_path = os.path.join(train_test_input_path, 'all')
    category_tables_list = [file for file in os.listdir(train_test_category_path) if file.endswith('.json.gz')]
    
    count = 0
    with progressbar.ProgressBar(max_value=len(valid_tables)) as bar:
        for table in category_tables_list:
            if os.path.isfile(os.path.join(train_test_all_path,'{}'.format(table))):
                all_table_df = pd.read_json(os.path.join(train_test_all_path,'{}'.format(table)), compression='gzip', lines=True)
                cat_table_df = pd.read_json(os.path.join(train_test_category_path,'{}'.format(table)), compression='gzip', lines=True)
                frames = [all_table_df, cat_table_df]
                all_table_df = pd.concat(frames)
            else:
                all_table_df = pd.read_json(os.path.join(train_test_category_path,'{}'.format(table)), compression='gzip', lines=True)
            all_table_df.to_json(os.path.join(train_test_all_path, '{}'.format(table)), compression='gzip', orient='records', lines=True)
            
            count += 1
            bar.update(count)

  0% (13 of 2280) |                      | Elapsed Time: 0:00:00 ETA:   0:00:18

Bikes


100% (2280 of 2280) |####################| Elapsed Time: 0:00:00 Time:  0:00:00
  1% (29 of 2280) |                      | Elapsed Time: 0:00:00 ETA:   0:00:14

Cars


100% (2280 of 2280) |####################| Elapsed Time: 0:00:01 Time:  0:00:01
  0% (16 of 2280) |                      | Elapsed Time: 0:00:00 ETA:   0:00:14

...

100% (2280 of 2280) |####################| Elapsed Time: 0:00:19 Time:  0:00:19
  0% (3 of 2280) |                       | Elapsed Time: 0:00:00 ETA:   0:01:21

Technology


100% (2280 of 2280) |####################| Elapsed Time: 0:00:08 Time:  0:00:08
100% (2280 of 2280) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


Tools
Random


100% (2280 of 2280) |####################| Elapsed Time: 0:00:28 Time:  0:00:28
